In [271]:
import pandas as pd
import numpy as np
import random 
data_path = "./cb_db_csv/"

In [2]:
pd.__version__

'0.23.4'

# cb_objects

In [11]:
df_object = pd.read_csv(data_path + 'cb_objects.csv', encoding = "ISO-8859-1")
df_object = df_object.replace(r'\N', np.nan)
df_object.founded_at = pd.to_datetime(df_object.founded_at)
df_object=df_object[pd.notnull(df_object['founded_at'])]
entity_type = ['Company']
df_object=df_object[df_object['entity_type'].isin(entity_type)]
df_object.category_code.unique()

17118087


array(['web', 'network_hosting', 'games_video', 'advertising',
       'cleantech', 'enterprise', 'other', 'consulting', 'mobile',
       'health', 'software', 'analytics', 'finance', 'education',
       'medical', 'manufacturing', 'ecommerce', 'public_relations',
       'hardware', nan, 'search', 'news', 'government', 'security',
       'biotech', 'photo_video', 'travel', 'semiconductor', 'social',
       'legal', 'transportation', 'hospitality', 'sports', 'nonprofit',
       'fashion', 'messaging', 'music', 'automotive', 'design',
       'real_estate', 'local', 'nanotech', 'pets'], dtype=object)

## Helper Method

## IPOs

In [389]:
df_ipo = pd.read_csv(data_path + 'cb_ipos.csv', encoding = "ISO-8859-1")
df_ipo = df_ipo.drop(['source_url','updated_at'],axis=1)
df_ipo = df_ipo.replace(r'\N', np.nan)
## drop object_id = null
## drop companies without id record
df_ipo = df_ipo[df_ipo.object_id.notnull()]

df_ipo = pd.merge(df_ipo,df_object[['id','founded_at']],how='left',left_on=['object_id'],right_on=['id'])


## make timestamp 
df_ipo.public_at = pd.to_datetime(df_ipo.public_at)
df_ipo.public_at = pd.to_datetime(df_ipo.created_at)
df_ipo['time_since_founded'] = pd.Timedelta('nan')
null_columns = df_ipo.columns[df_ipo.isnull().any()]
print(df_ipo[null_columns].isnull().sum())
df_ipo.shape

valuation_amount           1146
valuation_currency_code       2
raised_amount              1115
raised_currency_code        558
id_y                        547
founded_at                  547
time_since_founded         1254
dtype: int64


(1254, 13)

- Before: 1254
- Drop non-existing company id : 707


In [390]:
def getIPODuration(row):
    if pd.isnull(row['public_at']):
        row['public_at'] = row['created_at']
    if pd.notnull(row['founded_at']) and pd.notnull(row['public_at']):
        duration = row['public_at'] - row.founded_at
        row['time_since_founded'] = pd.to_timedelta([duration]).astype('timedelta64[D]').values
    return row
df_ipo = df_ipo.apply(getIPODuration, axis=1)
df_ipo = df_ipo[df_ipo['time_since_founded'].notnull()]
df_ipo = df_ipo.drop(['id_x','id_y','ipo_id','valuation_amount','valuation_currency_code','raised_amount','raised_currency_code',
                      'public_at','stock_symbol','created_at','founded_at'],axis=1)
df_ipo.sort_values(by=['time_since_founded'],inplace=True)
df_ipo = df_ipo[df_ipo['time_since_founded'].gt(-0.1)]

In [391]:
df_ipo

,object_id,time_since_founded
27,c:4495,[1]
908,c:144059,[67]
11,c:475,[83]
198,c:10614,[121]
806,c:164413,[324]
691,c:145411,[340]
1224,c:281988,[694]
1253,c:235945,[711]
716,c:48759,[754]
611,c:22130,[896]


In [392]:
df_ipo.to_pickle('./dataframe_cleaned/df_ipo') 

## Acquisition
### Acquistion date: created date 28 companies

In [ ]:
df_acq = pd.read_csv(data_path + 'cb_acquisitions.csv', encoding = "ISO-8859-1")
df_acq = df_acq.drop(['source_url','updated_at'],axis=1)

df_acq = df_acq.replace(r'\N', np.nan)
df_acq = df_acq[df_acq['acquired_object_id'].notnull()]

## Add acquistion data
df_acq.loc[df_acq['acquired_at'].isnull(),'acquired_at'] = df_acq.loc[df_acq['acquired_at'].isnull(),'created_at']
null_columns=df_acq.columns[df_acq.isnull().any()]
print(df_acq[null_columns].isnull().sum())


df_acq = pd.concat([df_acq, pd.get_dummies(df_acq['price_currency_code'])], axis=1, sort=False)
df_acq = pd.concat([df_acq, pd.get_dummies(df_acq['term_code'])], axis=1, sort=False)
df_acq = pd.merge(df_acq,df_object[['id','category_code']],how='left',left_on=['acquired_object_id'],right_on=['id'])

df_acq

In [20]:
## Acquiring
df_acq.acquired_at = pd.to_datetime(df_acq.acquired_at)
df_acq['acquiring_time_since_founded'] = pd.Timedelta('nan')
def getAcquiringDuration(row):
    df_row = df_object.loc[df_object['id'] == row['acquiring_object_id']]
    if not df_row.empty:
        if pd.notnull(df_row['founded_at'].values) and pd.notnull(row['acquired_at']):
            duration = row['acquired_at'] - df_row['founded_at']
            row['acquiring_time_since_founded'] = duration.astype('timedelta64[D]').values
    return row
df_acq = df_acq.apply(getAcquiringDuration, axis=1)


In [21]:
## Acquired
df_acq['acquired_time_since_founded'] = pd.Timedelta('nan')
def getAcquiredDuration(row):
    df_row = df_object.loc[df_object['id'] == row['acquired_object_id']]
    if not df_row.empty:
        if pd.notnull(df_row['founded_at'].values) and pd.notnull(row['acquired_at']):
            duration = row['acquired_at'] - df_row['founded_at']
            row['acquired_time_since_founded'] = duration.astype('timedelta64[D]').values
    return row
df_acq = df_acq.apply(getAcquiredDuration, axis=1)


In [22]:
df_acq.to_pickle('./dataframe_cleaned/df_acq') 
df_acq

,id_x,acquisition_id,acquiring_object_id,acquired_object_id,term_code,price_amount,price_currency_code,acquired_at,created_at,AUD,...,JPY,SEK,USD,cash,cash_and_stock,stock,id_y,category_code,acquiring_time_since_founded,acquired_time_since_founded
0,1,1,c:11,c:10,NaN,20000000,USD,2007-05-30,2007-05-31 22:19:54,0,...,0,0,1,0,0,0,NaN,NaN,[10225.0],NaT
1,2,7,c:59,c:72,cash,60000000,USD,2007-07-01,2007-07-03 08:14:50,0,...,0,0,1,1,0,0,c:72,mobile,[3219.0],[456.0]
2,3,8,c:24,c:132,cash,280000000,USD,2007-05-01,2007-07-12 04:19:24,0,...,0,0,1,1,0,0,c:132,music,[7790.0],[1946.0]
3,4,9,c:59,c:155,cash,100000000,USD,2007-06-01,2007-07-13 09:52:59,0,...,0,0,1,1,0,0,c:155,web,[3189.0],[1216.0]
4,5,10,c:212,c:215,cash,25000000,USD,2007-07-01,2007-07-20 05:29:07,0,...,0,0,1,1,0,0,c:215,web,[4929.0],[2860.0]
5,6,11,c:227,c:228,NaN,7140000,USD,2003-08-05,2007-07-20 09:58:29,0,...,0,0,1,0,0,0,c:228,web,[7078.0],[1496.0]
6,7,12,c:5,c:230,cash,NaN,USD,2007-07-01,2007-07-26 07:36:07,0,...,0,0,1,1,0,0,NaN,NaN,[1246.0],NaT
7,8,14,c:318,c:314,cash,NaN,USD,2006-10-01,2007-07-31 07:38:36,0,...,0,0,1,1,0,0,c:314,web,[-273.0],[638.0]
8,9,15,c:20,c:333,stock,1500000000,USD,2002-07-08,2007-08-01 09:58:26,0,...,0,0,1,0,0,1,c:333,finance,[2502.0],[1315.0]
9,10,16,c:342,c:341,cash,350000000,USD,2007-08-01,2007-08-02 09:12:27,0,...,0,0,1,1,0,0,c:341,games_video,[30605.0],[669.0]


In [149]:
## Acquiring

df_acquiring = df_acq.set_index(['acquiring_object_id','acquired_object_id']).sort_index()
df_acquiring.sort_values(by=['acquiring_time_since_founded'],inplace=True)
df_acquiring.drop(['id_x','acquisition_id','acquired_at','acquired_time_since_founded','id_y'],axis=1,inplace=True)
df_acquiring = df_acquiring[df_acquiring['acquiring_time_since_founded'].notnull()]
df_acquiring = df_acquiring[df_acquiring['acquiring_time_since_founded'].gt(0)]

## category average price

In [150]:
category = df_acquiring[df_acquiring['category_code'].notnull()]['category_code'].unique()
avg_price = []
for ctg in category:
    row = df_acquiring.loc[df_acquiring["category_code"]==ctg,['price_amount']]
    price = row[pd.notnull(row.price_amount)].astype('float128').mean()
    avg_price.append(price)
price_dic = dict(zip(category,avg_price))

In [152]:
def getPrice(row):
    if row[['price_amount']].isnull().item() and row[['category_code']].notnull().item():
        row[['price_amount']] = price_dic[row['category_code']]
    return row
df_acquiring = df_acquiring.apply(getPrice,axis=1)
df_acquiring = df_acquiring.dropna(subset=['category_code','price_amount'],how='all')

In [153]:
df_acquiring = df_acquiring[df_acquiring['price_amount'].notnull()]

df_acquiring = df_acquiring.drop(['price_currency_code','term_code','created_at','category_code'],axis=1)

In [166]:
df_acquiring


,,price_amount,AUD,CAD,EUR,GBP,JPY,SEK,USD,cash,cash_and_stock,stock,acquiring_time_since_founded
acquiring_object_id,acquired_object_id,,,,,,,,,,,,
c:260783,c:80142,2.40649e+08,0,0,0,0,0,0,1,0,0,0,[1.0]
c:307,c:306,2.39394e+08,0,0,0,0,0,0,1,1,0,0,[6.0]
c:60326,c:146208,1000000,0,0,0,0,0,0,1,0,0,1,[14.0]
c:15461,c:15460,70000,0,0,0,0,0,0,1,1,0,0,[25.0]
c:192899,c:193355,330000,0,0,0,0,0,0,1,0,1,0,[27.0]
c:138299,c:141275,15000,0,0,0,0,0,0,1,1,0,0,[29.0]
c:80259,c:4716,2.40649e+08,0,0,0,0,0,0,1,0,0,0,[31.0]
c:166448,c:40101,2.12816e+08,0,0,0,0,0,0,1,0,0,0,[31.0]
c:82144,c:60638,3.4151e+08,0,0,0,0,0,0,1,0,0,0,[39.0]


In [160]:
df_acquiring.to_pickle('./dataframe_cleaned/df_acquiring') 

Before kicking away the negative durations 
- 9532 rows × 4 columns 

After kicking away the negative durations 
- 6866 rows × 4 columns

After dropping entries with missing values price amount
- 1875 rows × 4 columns

Current:
- 4130 rows

## Acquired

Before kicking away the negative durations 
- 9532 rows × 4 columns

After kicking away the negative durations 
- 4821 rows × 4 columns


In [165]:
df_acquired = df_acq.set_index(['acquired_object_id']).sort_index()
df_acquired.sort_values(by=['acquired_time_since_founded'],inplace=True)
df_acquired.drop(['id_x','id_y','created_at','category_code','acquisition_id','acquiring_object_id','acquired_at','acquiring_time_since_founded','term_code','price_amount','price_currency_code'],axis=1,inplace=True)
df_acquired = df_acquired[df_acquired['acquired_time_since_founded'].gt(0)]

df_acquired.to_pickle('./dataframe_cleaned/df_acquired') 
df_acquired

,AUD,CAD,EUR,GBP,JPY,SEK,USD,cash,cash_and_stock,stock,acquired_time_since_founded
acquired_object_id,,,,,,,,,,,
c:22103,0,0,0,0,0,0,1,1,0,0,[18.0]
c:54651,0,0,0,0,0,0,1,0,0,0,[29.0]
c:159552,0,0,0,0,0,0,1,0,0,1,[31.0]
c:329,0,0,0,0,0,0,1,0,0,0,[31.0]
c:2315,0,0,0,0,0,0,1,0,0,0,[31.0]
c:54405,0,0,0,0,0,0,1,0,0,0,[59.0]
c:8565,0,0,0,0,0,0,1,0,0,0,[67.0]
c:86,0,0,0,0,0,0,1,0,0,0,[67.0]
c:45891,0,0,0,0,0,0,1,1,0,0,[75.0]


## Funding


## Funding Data
- Initial: 17689 companies 31954 rows
- After: 16778 companies , 25854 rows 

In [350]:
TermList = ['angel','seed','a', 'b','c','d','e','f','g']
            # 'convertible','crowd','crowd_equity','debt_round','grant',
            # 'partial','post_ipo_debt','post_ipo_equity','private_equity','secondary_market','unattributed']


In [351]:
df_funding= pd.read_csv(data_path + 'cb_funding_rounds.csv', encoding = "ISO-8859-1")

df_funding = df_funding.replace(r'\N', np.nan)


#drop those funding rounds out of concern
df_funding = df_funding[df_funding['funding_round_code'].isin(TermList)]
df_funding = pd.concat([df_funding, pd.get_dummies(df_funding['funding_round_code'])], axis=1, sort=False)

df_funding = pd.merge(df_funding,df_object[['id','founded_at']],how='left',left_on=['object_id'],right_on=['id'])
df_funding = df_funding.drop(['source_url','created_by','pre_money_valuation_usd',
                            'pre_money_valuation','pre_money_currency_code','post_money_valuation_usd',
                             'post_money_valuation','post_money_currency_code','raised_amount','raised_currency_code'], axis=1)
df_funding

df_funding = df_funding[df_funding['founded_at'].notnull()]

df_funding['created_at'] =  pd.to_datetime(df_funding['created_at'])
df_funding.funded_at = pd.to_datetime(df_funding.funded_at)

In [352]:

def getFundDate(row):
    if row[['funded_at']].isnull().item():
        if row[['is_last_round']].item()==1 or (row[['funding_round_code']] == 'angel').item()==1:
            row[['funded_at']] = row[['founded_at']]
        elif row[['is_first_round']].item()==1 or (row[['funding_round_code']] == 'a').item()==1:
            row[['funded_at']] = row[['created_at']]
        elif (row[['funding_round_code']] =='seed').item()==1:
            pre_rows = df_funding[df_funding['object_id'] == row['object_id']]
            angel_row = pre_rows[pre_rows['funding_round_code']=='angel']
            if not angel_row.empty and angel_row['funded_at'].notnull().item()==1:
                td = row['created_at'] - angel_row['funded_at']
                td = td.astype('timedelta64[D]').values
                delta = random.randint(1,td)
                row[['funded_at']] = angel_row[['funded_at']] + pd.DateOffset(days=delta)
            else:
                row[['funded_at']] = row[['created_at']]
        elif (row[['funding_round_code']] =='b').item()==1:
            pre_rows = df_funding[df_funding['object_id'] == row['object_id']]
            angel_row = pre_rows[pre_rows['funding_round_code']=='a']
            if not angel_row.empty and angel_row['funded_at'].notnull().item()==1:
                td = row['created_at'] - angel_row['funded_at']
                td = td.astype('timedelta64[D]').values
                delta = random.randint(1,td)
                row[['funded_at']] = angel_row[['funded_at']] + pd.DateOffset(days=delta)
            else:
                row[['funded_at']] = row[['created_at']]
        elif (row[['funding_round_code']] =='c').item()==1:
            pre_rows = df_funding[df_funding['object_id'] == row['object_id']]
            angel_row = pre_rows[pre_rows['funding_round_code']=='b']
            if not angel_row.empty and angel_row['funded_at'].notnull().item()==1:
                td = row['created_at'] - angel_row['funded_at']
                td = td.astype('timedelta64[D]').values
                delta = random.randint(1,td)
                row[['funded_at']] = angel_row[['funded_at']] + pd.DateOffset(days=delta)
            else:
                row[['funded_at']] = row[['created_at']]
    return row

df_funding = df_funding.apply(getFundDate,axis=1)
                
                
                
                
            

In [353]:
## make timestamp 

df_funding['time_since_founded'] = pd.Timedelta('nan')

def getDuration(row):
    
    #df_row = df_object.loc[df_object['id'] == row.object_id]
    #if not df_row.empty:
    if pd.notnull(row['founded_at']) and pd.notnull(row['funded_at']):
    #        duration = row.funded_at - df_row.founded_at
        duration = row.funded_at - row.founded_at
        row['time_since_founded'] = pd.to_timedelta([duration]).astype('timedelta64[D]').values
    return row

df_funding = df_funding.apply(getDuration,axis=1)

#only positive date
df_funding = df_funding[df_funding['time_since_founded'].gt(-0.1)]
df_funding['time_since_founded'] = df_funding['time_since_founded'].astype(int)

In [354]:
df_funding.sort_values(by=['time_since_founded'],inplace=True)
df_funding

,id_x,funding_round_id,object_id,funded_at,funding_round_type,funding_round_code,raised_amount_usd,participants,is_first_round,is_last_round,...,b,c,d,e,f,g,seed,id_y,founded_at,time_since_founded
16944,26956,26956,c:144040,2008-03-01,angel,seed,1000000,0,1,1,...,0,0,0,0,0,0,1,c:144040,2008-03-01,0
4093,4962,4962,c:18038,2006-09-01,series-a,a,13350000,4,0,1,...,0,0,0,0,0,0,0,c:18038,2006-09-01,0
19165,30642,30642,c:161452,2012-01-01,angel,seed,NaN,0,1,1,...,0,0,0,0,0,0,1,c:161452,2012-01-01,0
19166,30643,30643,c:161453,2012-01-01,angel,seed,100000,0,1,1,...,0,0,0,0,0,0,1,c:161453,2012-01-01,0
19169,30648,30648,c:161480,2012-02-01,angel,angel,250000,1,1,1,...,0,0,0,0,0,0,0,c:161480,2012-02-01,0
28530,48635,48635,c:264245,2009-07-01,angel,angel,NaN,1,0,1,...,0,0,0,0,0,0,0,c:264245,2009-07-01,0
4121,5004,5004,c:18202,2008-04-01,angel,seed,425000,2,0,1,...,0,0,0,0,0,0,1,c:18202,2008-04-01,0
19187,30677,30677,c:51578,2009-01-01,angel,angel,695850,0,0,1,...,0,0,0,0,0,0,0,c:51578,2009-01-01,0
19191,30683,30683,c:161699,2011-06-01,angel,seed,575000,1,1,1,...,0,0,0,0,0,0,1,c:161699,2011-06-01,0
19212,30717,30717,c:161944,2012-03-01,angel,angel,1000000,5,1,1,...,0,0,0,0,0,0,0,c:161944,2012-03-01,0


## Average fund each term

In [355]:

avg_fund = []
for term in TermList:
    row = df_funding.loc[df_funding['funding_round_code']==term,['raised_amount_usd']]
    fund = row[pd.notnull(row.raised_amount_usd)].astype('float128').mean()
    avg_fund.append(fund)
fund_dic = dict(zip(TermList,avg_fund))

def getFund(row):
    if row[['raised_amount_usd']].isnull().item() and row[['funding_round_code']].notnull().item():
        row[['raised_amount_usd']] = fund_dic[row['funding_round_code']]
    return row
df_funding = df_funding.apply(getFund,axis=1)

In [356]:

df_funding.drop(['id_x','id_y','created_at','updated_at','funding_round_id','funding_round_type','founded_at','funded_at','is_first_round','is_last_round'],axis=1,inplace=True)


In [359]:
df_funding.to_pickle('./dataframe_cleaned/df_funding')

In [331]:
len(df_funding.groupby('object_id').groups)

16778

In [360]:
null_columns = df_funding.columns[df_funding.isnull().any()]
print(df_funding[null_columns].isnull().sum())
df_funding

Series([], dtype: float64)


,object_id,funding_round_code,raised_amount_usd,participants,a,angel,b,c,d,e,f,g,seed,time_since_founded
16944,c:144040,seed,1000000,0,0,0,0,0,0,0,0,0,1,0
4093,c:18038,a,13350000,4,1,0,0,0,0,0,0,0,0,0
19165,c:161452,seed,866033,0,0,0,0,0,0,0,0,0,1,0
19166,c:161453,seed,100000,0,0,0,0,0,0,0,0,0,1,0
19169,c:161480,angel,250000,1,0,1,0,0,0,0,0,0,0,0
28530,c:264245,angel,832099,1,0,1,0,0,0,0,0,0,0,0
4121,c:18202,seed,425000,2,0,0,0,0,0,0,0,0,1,0
19187,c:51578,angel,695850,0,0,1,0,0,0,0,0,0,0,0
19191,c:161699,seed,575000,1,0,0,0,0,0,0,0,0,1,0
19212,c:161944,angel,1000000,5,0,1,0,0,0,0,0,0,0,0
